In [1]:
import numpy as np # primary math library
import matplotlib.pyplot as plt # primary plotting library
%matplotlib inline

import nelpy as nel # should be installed using pip install nelpy

import warnings
#warnings.filterwarnings("ignore") # useful to prevent excess nelpy warnings

In [2]:
import os

dataroot = '/Users/ckemere/Development/Data/Frank/';

filename = os.path.join(dataroot,'FrankLabData.nel')
import pandas as pd
import nelpy.io
jar = nel.load_pkl(filename)

data = jar.data
tetinfo = jar.tetinfo
cellinfo = jar.cellinfo
taskinfo = jar.taskinfo
del jar

In [3]:
def detect_ripples(eeg):
    """Detect sharp wave ripples (SWRs) from single channel eeg (AnalogSignalArray).
    """
    
    # Maggie defines ripples by doing:
    #  (1) filter 150-250
    #  (2) hilbert envelope
    #  (3) smooth with Gaussian (4 ms SD)
    #  (4) 3.5 SD above the mean for 15 ms
    #  (5) full ripple defined as window back to mean

    assert eeg.n_signals == 1, "only single channel ripple detection currently supported!"
    
    # (1)
    ripple_eeg = nel.filtering.sosfiltfilt(eeg, fl=150, fh=250)
    # (2, 3)
    ripple_envelope = nel.utils.signal_envelope1D(ripple_eeg, sigma=0.004)
    # (4, 5)
    bounds, maxes, events = nel.utils.get_events_boundaries(
                x=ripple_envelope.ydata,
                PrimaryThreshold=ripple_envelope.mean() + 3.5*ripple_envelope.std(),   # cm/s
                SecondaryThreshold=ripple_envelope.mean(),  # cm/s
                minThresholdLength=0.015, # threshold crossing must be at least 15 ms long
                minLength=0.0, # total ripple duration must be at least XXX ms long
                ds = 1/ripple_envelope.fs
            )

    # convert bounds (in samples) to time in seconds
    timebounds = ripple_envelope.time[bounds]

    # add 1/fs to stops for open interval
    timebounds[:,1] += 1/eeg.fs

    # create EpochArray with bounds
    ripple_epochs = nel.EpochArray(timebounds)
    return ripple_epochs
    

In [12]:
import frank_lab # Caleb's loading routines; modified a bit to support verbose keyword consistently,
                 # and for dealing with known Con gotcha in Windows 
                 # (see e.g. https://fossbytes.com/windows-reserved-folder-con-create/)

In [13]:
import os
anim_dirs = [(d[:3], os.path.join(dataroot,d)) for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot,d))]
print(anim_dirs)

[('Bon', '/Users/ckemere/Development/Data/Frank/Bon'), ('Con', '/Users/ckemere/Development/Data/Frank/Con'), ('Eig', '/Users/ckemere/Development/Data/Frank/Eig'), ('Fra', '/Users/ckemere/Development/Data/Frank/Fra'), ('Ten', '/Users/ckemere/Development/Data/Frank/Ten')]


In [43]:
RipThresh = 3 # x sigma over mean
minThresholdLength = 0.015
minLength = 0.025
maxLength = 2

ani, animal_dir = 'Bon', '/Users/ckemere/Development/Data/Frank/Bon'


# import os.path
# filename = os.path.join('LFP','LFP_{}_{:02d}_{:02d}.nel'.format(anim,day,tetrode))
# print('Loading: {}'.format(filename))
# if os.path.exists(filename):
#     lfpjar = nel.load_pkl(filename)
# else:
#     print('No LFP file for {}'.format(filename))
#     continue

all_swr_events = {}
for day, daydata in data[anim].items():
    # The animal "Bon" had good data on day 3
    S = daydata['spikes']
    pos = daydata['pos']
    ep_list = list(taskinfo.query('Animal==@anim & Day==@dk')["Epoch"])
    supports = []
    day_swr_events = []
    for tet in tetrodes:
        tet_eeg = []
        eeg_timestamps = []
        for ep in ep_list:
            tetrodes = tetinfo.query('Animal==@anim & Epoch==@ep & area=="CA1" & Day==@day').Tetrode.tolist()
            eeg = frank_lab.load_data(animal_dir, day=day+1, epoch=ep+1, 
                                      tetrode=tet+1, version='new', verbose=True)

            start = eeg.starttime
            stop = eeg.starttime + len(eeg.data) / eeg.samprate

            tet_eeg.append(eeg.data)
            eeg_timestamps.append(eeg.starttime + np.array(range(len(eeg.data))) / eeg.samprate)
        
            # might as well build one AnalogSignalArray for the eeg data:
        eeg = nel.AnalogSignalArray(ydata=np.hstack(tet_eeg), timestamps=np.hstack(eeg_timestamps), fs=1500)
        ripple_epochs = detect_ripples(eeg)
        day_swr_events.append(ripple_epochs)
    
    break;
    
    # lfpdata = lfpjar.lfpdata
    # ripple_epochs = detect_ripples(lfpdata)
    # all_swr_events[day] = ripple_epochs

Loading boneeg03-1-11.mat
Loading boneeg03-2-11.mat
Loading boneeg03-3-11.mat
Loading boneeg03-4-11.mat
Loading boneeg03-5-11.mat
Loading boneeg03-6-11.mat
Loading boneeg03-7-11.mat


/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:360: UserWarning: creating support from time and sampling rate, fs!
/Users/ckemere/Development/Python/nelpy/nelpy/utils.py:549: UserWarning: some steps in the data are smaller than the requested step size.


Loading boneeg03-1-12.mat
Loading boneeg03-2-12.mat
Loading boneeg03-3-12.mat
Loading boneeg03-4-12.mat
Loading boneeg03-5-12.mat
Loading boneeg03-6-12.mat
Loading boneeg03-7-12.mat


/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:360: UserWarning: creating support from time and sampling rate, fs!
/Users/ckemere/Development/Python/nelpy/nelpy/utils.py:549: UserWarning: some steps in the data are smaller than the requested step size.


Loading boneeg03-1-13.mat
Loading boneeg03-2-13.mat
Loading boneeg03-3-13.mat
Loading boneeg03-4-13.mat
Loading boneeg03-5-13.mat
Loading boneeg03-6-13.mat
Loading boneeg03-7-13.mat


...

/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:360: UserWarning: creating support from time and sampling rate, fs!
/Users/ckemere/Development/Python/nelpy/nelpy/utils.py:549: UserWarning: some steps in the data are smaller than the requested step size.


Loading boneeg03-1-04.mat
Loading boneeg03-2-04.mat
Loading boneeg03-3-04.mat
Loading boneeg03-4-04.mat
Loading boneeg03-5-04.mat
Loading boneeg03-6-04.mat
Loading boneeg03-7-04.mat


/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:360: UserWarning: creating support from time and sampling rate, fs!
/Users/ckemere/Development/Python/nelpy/nelpy/utils.py:549: UserWarning: some steps in the data are smaller than the requested step size.


Loading boneeg03-1-05.mat
Loading boneeg03-2-05.mat
Loading boneeg03-3-05.mat
Loading boneeg03-4-05.mat
Loading boneeg03-5-05.mat
Loading boneeg03-6-05.mat
Loading boneeg03-7-05.mat


/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:360: UserWarning: creating support from time and sampling rate, fs!
/Users/ckemere/Development/Python/nelpy/nelpy/utils.py:549: UserWarning: some steps in the data are smaller than the requested step size.


In [40]:
for e in ep_eeg:
    print(e.shape)

(1689946,)
(1689946,)
(1689946,)
(1689946,)
(1689946,)
(1689946,)
(1689943,)
(1689943,)
(1689943,)
(1689946,)
(1689946,)


In [28]:
tetrodes = tetinfo.query('Animal==@anim & area=="CA1" & Day==@day').Tetrode.tolist()
tets = [t+1 for t in tetrodes]

eeg = frank_lab.load_data(animal_dir, day=day+1, epoch=ep+1, 
                          tetrode=tets, version='new', verbose=True)



TypeError: unsupported format string passed to list.__format__

In [23]:
import nelpy.io
#np.savez_compressed('FrankLabPickle.npz', data=data, cellinfo=cellinfo, taskinfo=taskinfo)
BonRippleEpochs = nel.ResultsContainer(BonSWREpochs=all_swr_events,
                                    description='SWR epochs detected in Bon data using 5 sigma threshold.\n')

In [24]:
BonRippleEpochs.save_pkl('BonRippleEpochs.nel')